In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import DatastreamDSWS as dsws
ds=dsws.Datastream(username='ben.mcdonald@omg.ca',password='Uuddlrlrba55')

In [42]:
daily_data = ds.get_data(tickers='MSACWFL(MSRI), MLUS710(RI), MLCORML(RI), MLHMACL(RI)',
                         fields=['PCH#(X,-1D)'], start='2002-01-01', end='0D', freq='D')
daily_data.index = pd.to_datetime(daily_data.index)
daily_data = daily_data.rolling(window=5).sum().dropna()

In [43]:
monthly_data = ds.get_data(tickers='MSACWFL(MSRI), MLUS710(RI), MLCORML(RI), MLHMACL(RI), MSERCAD, MLCR1CL(RI)',
                           fields=['PCH#(X,-1M)'], start='2002-01-31', end='0D', freq='M')
monthly_data.index = pd.to_datetime(monthly_data.index)

In [44]:
#Saves daily data columns into series
MKT = daily_data.iloc[:,0]
INT = daily_data.iloc[:,1]
IG = daily_data.iloc[:,2]
HY = daily_data.iloc[:,3]

In [45]:
#Finds last trading day in each month
month_end_dates = []
for d in monthly_data.index:
    
    while d not in daily_data.index:
        d - pd.Timedelta(days=1)
        
    month_end_dates.append(d)

In [46]:
#Sets parameters for weighted regression including trailing period (3 years) and half life (1 year of trading days)
wls = LinearRegression()
rolling_months = 36
halflife = 260
ig_betas = []
hy_betas = []

for i in range(len(month_end_dates)-rolling_months):  #For all monthly 3 year windows in time period
    
    start_date = month_end_dates[i]                                  #Sets the start date
    end_date = month_end_dates[i+rolling_months-1]                   #Sets the end date
    rolling_days = daily_data.loc[start_date : end_date].shape[0]    #Sets the number of days in the 
    
    #Finds the MKT and INT beta in Investment Grade returns
    ig_beta = wls.fit(pd.concat([MKT, INT], axis=1).loc[start_date : end_date],
                      IG.loc[start_date : end_date],
                      sample_weight = [(0.5 ** (1 / halflife)) ** i for i in range(rolling_days)][::-1]).coef_
    
    #Finds the MKT and INT beta in High Yield returns
    hy_beta = wls.fit(pd.concat([MKT, INT], axis=1).loc[start_date : end_date],
                      HY.loc[start_date : end_date],
                      sample_weight = [(0.5 ** (1 / halflife)) ** i for i in range(rolling_days)][::-1]).coef_
    
    ig_betas.append(ig_beta)  #Appends IG betas to list
    hy_betas.append(hy_beta)  #Appends HY betas to list

#Turns lists into arrays for flexibility
ig_betas = np.array(ig_betas)
hy_betas = np.array(hy_betas)

In [47]:
#Saves monthly data columns into series
monthly_data = monthly_data.iloc[rolling_months:]
MKT = monthly_data.iloc[:,0]
INT = monthly_data.iloc[:,1]
IG = monthly_data.iloc[:,2]
HY = monthly_data.iloc[:,3]
CUR = monthly_data.iloc[:,4]
RF = monthly_data.iloc[:,5]

In [48]:
#Creates a list of Investment Grade's MKT betas and INT betas
ig_mkt_crd_betas = [b[0] for b in ig_betas]
ig_int_crd_betas = [b[1] for b in ig_betas]

#Creates a list of High Yield's MKT betas and INT betas
hy_mkt_crd_betas = [b[0] for b in hy_betas]
hy_int_crd_betas = [b[1] for b in hy_betas]

In [49]:
#Removes MKT and INT beta returns from IG and HY returns 
IG = IG - MKT * ig_mkt_crd_betas - INT * ig_int_crd_betas
HY = HY - MKT * hy_mkt_crd_betas - INT * hy_int_crd_betas

#Takes average
CRD = 0.5*IG + 0.5*HY

In [50]:
monthly_data = pd.concat([MKT, INT, CRD, CUR, RF], axis=1)
monthly_data.columns = 'MKT INT CRD CUR RF'.split()

In [51]:
monthly_data.describe()

,MKT,INT,CRD,CUR,RF
count,191.000000,191.000000,191.000000,191.000000,191.000000
mean,0.734356,0.425304,0.136148,0.078815,0.147775
std,4.047326,1.714062,1.533485,2.772304,0.104911
min,-16.918000,-4.116000,-8.652566,-7.768740,0.022000
25%,-1.138000,-0.576000,-0.304993,-1.595215,0.079500
50%,1.395000,0.206000,0.185930,0.016040,0.102000
75%,2.900000,1.382500,0.574773,1.587050,0.200000
max,11.462000,7.995000,6.860947,14.357590,0.453000


In [52]:
monthly_data.corr()

,MKT,INT,CRD,CUR,RF
MKT,1.000000,-0.333773,0.383799,-0.640662,-0.171121
INT,-0.333773,1.000000,-0.225191,0.233498,0.111991
CRD,0.383799,-0.225191,1.000000,-0.389614,-0.196524
CUR,-0.640662,0.233498,-0.389614,1.000000,0.052216
RF,-0.171121,0.111991,-0.196524,0.052216,1.000000
